In [8]:
import numpy as np
import cv2
import tensorflow as tf
import pickle as pkl
from tensorflow.keras import models
from skimage.feature import hog
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
import math

In [2]:
def Load_array(name):
    with open(name + '.pkl', 'rb') as f:
        arr = pkl.load(f)
    return arr

In [3]:
#Divides height and width by scale factor
def single_window(img, clf):
    
    #Initializes Box
    box = [0,0,0,0]
    
    #Grabs cropped window to scale to 64x128 frame
    crop = img[0:img.shape[0], int(img.shape[1]/2-img.shape[0]/4):int(img.shape[1]/2+img.shape[0]/4)]
    
    #Scales image to 64x128
    window = cv2.resize(crop, dsize=(64, 128), interpolation=cv2.INTER_CUBIC)     
    
    #Grabs HOG features from 64x128 image
    HOG,_ =  hog(window, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2),visualize = True) #HistogramOfGradients(window)
    
    #SVM linear classifier to determine if a person was detected 
    score = np.dot(([HOG]),clf.coef_.T)+clf.intercept_
       
    #score > 1 if person and score < -1 if not a person
    if score >= 1:
        box = ([int(img.shape[1]/2-img.shape[0]/4), 0, int(img.shape[1]/2+img.shape[0]/4),img.shape[0]])       
    
    return box, score

In [4]:
def calculate_distance(color_frame,depth_frame, x1,y1,x2,y2):
    #Grabs depth of first and second coordinates
    print(y1)
    print(y2)
    color_intrin = color_frame.profile.as_video_stream_profile().intrinsics
    udist = depth_frame.get_distance(x1,y1)
    vdist = depth_frame.get_distance(x2, y2)

    #Gets 3d coordinate point of 2d picture coordinates
    point1 = rs.rs2_deproject_pixel_to_point(color_intrin, [x1, x1], udist)
    point2 = rs.rs2_deproject_pixel_to_point(color_intrin, [x2, y2], vdist)

    #Calculate distance of two coordinates in 3-dimensions
    dist = math.sqrt(
        math.pow(point1[0] - point2[0], 2) + math.pow(point1[1] - point2[1],2) + math.pow(
            point1[2] - point2[2], 2))

    return dist

In [5]:
def midpoint(x1,y1,x2,y2):
    
    x = (x1+x2)/2
    y = (y1+y2)/2

    return x,y

In [6]:
data = np.empty([1,228,228,3],dtype = int)
denormalize = np.empty([14,2],dtype = int)
x= np.empty([14,1],dtype = int)
y= np.empty([14,1],dtype = int)
radius = 8
green = (0,255,0)
red = (0, 0, 255) 
yellow = (255,255,0)
fill = -1
thickness = 5
font = cv2.FONT_HERSHEY_SIMPLEX 
boldness = 2
font_scale = 0.75

bw = 228
bh = 228
bc = 114
scale = float(720/228)
offset = 1280/2-720/2

In [10]:
try:
    
    #Required Header function for using camera--------------------------------------
    pipeline = rs.pipeline() 
    cfg = rs.config()
    profile = pipeline.start(cfg)
    #-------------------------------------------------------------------------------
    
    #Loading the linear classifier and neural network model-------------------------
    clf = Load_array('clf') #Detects humans in a picture 64x128 frame
    model = models.load_model('model2.h5') #Detects 14 joints which is 28 output coordinate points x and y
    #-------------------------------------------------------------------------------
    
    while True:
        #pyrealsense2 code to show frames--------------------------------------------
        
        #Grabs color frames
        frame = pipeline.wait_for_frames() #Grabs frames from camera
        color_frame = frame.get_color_frame()
        color = np.asanyarray(color_frame.get_data())

        #Align Color and Depth Frames
        align = rs.align(rs.stream.color)
        frame = align.process(frame)

        #Grabs depth frames:
        aligned_depth_frame = frame.get_depth_frame()
        depth = np.asanyarray(aligned_depth_frame.get_data())
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        depth = depth * depth_scale

        #machine vision code to detect person--------------------------------------
        
        box, score = single_window(color,clf)  
        if box[2]-box[0] > 0 and box[3] - box[1] > 0:
             
            #-----------------------------------neural net code--------------------
            crop = color[0:color.shape[0], int(color.shape[1]/2-color.shape[0]/2):int(color.shape[1]/2+color.shape[0]/2)]
            crop = cv2.resize(crop, dsize=(228, 228), interpolation=cv2.INTER_CUBIC) 
            data[0] = crop
            coordinates = model.predict(data)

            for i in range(0,14):
                denormalize[i,0] = (coordinates[0][2*i] * bw) + bc
                denormalize[i,1] = (coordinates[0][2*i + 1] * bh) + bc
                x[i] = int(denormalize[i,0]*scale+offset)
                y[i] = int(denormalize[i,1]*scale)

            for i in range (0,14):
                center_coordinates = (x[i], y[i])
                cv2.circle(color, center_coordinates, radius, red, fill) 

            cv2.line(color, (x[0],y[0]), (x[2],y[2]), red, thickness) #left foot to left knee
            cv2.line(color, (x[1],y[1]), (x[3],y[3]), red, thickness) #right foot to right knee
            cv2.line(color, (x[2],y[2]), (x[4],y[4]), red, thickness) #left knee to left upper thigh
            cv2.line(color, (x[3],y[3]), (x[5],y[5]), red, thickness) #right knee to right upper thigh
            cv2.line(color, (x[4],y[4]), (x[12],y[12]), red, thickness) #left upper thigh to neck
            cv2.line(color, (x[5],y[5]), (x[12],y[12]), red, thickness) #right upper thigh to neck
            cv2.line(color, (x[6],y[6]), (x[8],y[8]), red, thickness) #left hand to left elbow
            cv2.line(color, (x[7],y[7]), (x[9],y[9]), red, thickness) #right hand to right elbow
            cv2.line(color, (x[8],y[8]), (x[10],y[10]), red, thickness) #left elbow to left shoulder
            cv2.line(color, (x[9],y[9]), (x[11],y[11]), red, thickness) #right elbow to right shoulder
            cv2.line(color, (x[10],y[10]), (x[12],y[12]), red, thickness) #left shoulder to neck
            cv2.line(color, (x[11],y[11]), (x[12],y[12]), red, thickness) #right shoulder to neck
            cv2.line(color, (x[12],y[12]), (x[13],y[13]), red, thickness) #neck to top head
            #------------------------------------------------------------------------
            
            cv2.rectangle(color, (box[0], box[1]), (box[2], box[3]), green, 2)

            lfoot_lknee = calculate_distance(color_frame,aligned_depth_frame,x[0],y[0],x[2],y[2])
            rfoot_rknee = calculate_distance(color_frame,aligned_depth_frame,x[1],y[1],x[3],y[3])
            lknee_lhip = calculate_distance(color_frame,aligned_depth_frame,x[2],y[2],x[4],y[4])
            rknee_rhip = calculate_distance(color_frame,aligned_depth_frame,x[3],y[3],x[5],y[5])
            lhip_neck = calculate_distance(color_frame,aligned_depth_frame,x[4],y[4],x[12],y[12])
            rhip_neck = calculate_distance(color_frame,aligned_depth_frame,x[5],y[5],x[12],y[12])
            lhand_lelbow = calculate_distance(color_frame,aligned_depth_frame,x[6],y[6],x[8],y[8])
            rhand_relbow = calculate_distance(color_frame,aligned_depth_frame,x[7],y[7],x[9],y[9])
            lelbow_lshoulder = calculate_distance(color_frame,aligned_depth_frame,x[8],y[8],x[10],y[10])
            relbow_rshoulder = calculate_distance(color_frame,aligned_depth_frame,x[9],y[9],x[11],y[11])
            lshoulder_neck = calculate_distance(color_frame,aligned_depth_frame,x[10],y[10],x[12],y[12])
            rshoulder_neck = calculate_distance(color_frame,aligned_depth_frame,x[11],y[11],x[12],y[12])
            neck_head = calculate_distance(color_frame,aligned_depth_frame,x[12],y[12],x[13],y[13])
    
            #1 meter = 39.3701 inches
            cv2.putText(color, str(round(lfoot_lknee*39.3701)) + '"', midpoint(x[0],y[0],x[2],y[2]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(rfoot_rknee*39.3701)) + '"', midpoint(x[1],y[1],x[3],y[3]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(lknee_lhip*39.3701)) + '"', midpoint(x[2],y[2],x[4],y[4]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(rknee_rhip*39.3701)) + '"', midpoint(x[3],y[3],x[5],y[5]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(lhip_neck*39.3701)) + '"', midpoint(x[4],y[4],x[12],y[12]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(rhip_neck*39.3701)) + '"', midpoint(x[5],y[5],x[12],y[12]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(lhand_lelbow*39.3701)) + '"', midpoint(x[6],y[6],x[8],y[8]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(rhand_relbow*39.3701)) + '"', midpoint(x[7],y[7],x[9],y[9]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(lelbow_lshoulder*39.3701)) + '"', midpoint(x[8],y[8],x[10],y[10]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(relbow_rshoulder*39.3701)) + '"', midpoint(x[9],y[9],x[11],y[11]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(left_shoulder*39.3701)) + '"', midpoint(x[10],y[10],x[12],y[12]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(lshoulder_neck*39.3701)) + '"', midpoint(x[11],y[11],x[12],y[12]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(rshoulder_neck*39.3701)) + '"', midpoint(x[11],y[11],x[12],y[12]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
            cv2.putText(color, str(round(neck_head*39.3701)) + '"', midpoint(x[12],y[12],x[13],y[13]), font, font_scale, yellow, boldness, cv2.LINE_AA) 
    
    
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', color)
        if cv2.waitKey(40) & 0xFF == ord('q'):
            break
        
    cv2.destroyAllWindows()
    cv2.waitKey(1)        

finally:
    pipeline.stop()

[694]
[558]
[707]
[562]
[558]
[420]
[562]
[423]
[420]
[157]
[423]
[157]
[271]
[249]
[337]
[271]
[249]
[170]
[271]
[164]
[170]
[157]
[164]
[157]
[157]
[41]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[157]
[420]
[157]
[325]
[268]
[353]
[274]
[268]
[176]
[274]
[170]
[176]
[157]
[170]
[157]
[157]
[44]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[157]
[420]
[157]
[325]
[268]
[353]
[274]
[268]
[176]
[274]
[170]
[176]
[157]
[170]
[157]
[157]
[41]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[416]
[420]
[157]
[416]
[157]
[325]
[268]
[353]
[274]
[268]
[176]
[274]
[170]
[176]
[157]
[170]
[157]
[157]
[41]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[157]
[420]
[157]
[325]
[268]
[356]
[274]
[268]
[173]
[274]
[170]
[173]
[157]
[170]
[157]
[157]
[41]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[157]
[420]
[157]
[318]
[265]
[356]
[274]
[265]
[173]
[274]
[170]
[173]
[157]
[170]
[157]
[157]
[44]
[682]
[549]
[694]
[549]
[549]
[420]
[549]
[420]
[420]
[157]
[420]
[157

[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[161]
[416]
[161]
[350]
[281]
[360]
[277]
[281]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[47]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[341]
[277]
[356]
[277]
[277]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[44]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[337]
[274]
[353]
[277]
[274]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[44]
[675]
[546]
[688]
[549]
[546]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[47]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[341]
[277]
[356]
[277]
[277]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[44]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[337]
[274]
[353]
[277]
[274]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[44]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161

[685]
[552]
[694]
[552]
[552]
[416]
[552]
[420]
[416]
[154]
[420]
[154]
[281]
[249]
[341]
[265]
[249]
[167]
[265]
[164]
[167]
[154]
[164]
[154]
[154]
[34]
[688]
[552]
[697]
[552]
[552]
[416]
[552]
[420]
[416]
[151]
[420]
[151]
[274]
[246]
[341]
[265]
[246]
[167]
[265]
[164]
[167]
[151]
[164]
[151]
[151]
[34]
[685]
[552]
[697]
[552]
[552]
[416]
[552]
[420]
[416]
[151]
[420]
[151]
[274]
[246]
[341]
[265]
[246]
[167]
[265]
[164]
[167]
[151]
[164]
[151]
[151]
[34]
[685]
[552]
[697]
[552]
[552]
[416]
[552]
[420]
[416]
[151]
[420]
[151]
[274]
[246]
[341]
[265]
[246]
[167]
[265]
[164]
[167]
[151]
[164]
[151]
[151]
[34]
[685]
[552]
[697]
[552]
[552]
[416]
[552]
[420]
[416]
[154]
[420]
[154]
[284]
[249]
[344]
[268]
[249]
[167]
[268]
[167]
[167]
[154]
[167]
[154]
[154]
[37]
[685]
[552]
[697]
[552]
[552]
[420]
[552]
[420]
[420]
[154]
[420]
[154]
[284]
[249]
[344]
[268]
[249]
[167]
[268]
[167]
[167]
[154]
[167]
[154]
[154]
[37]
[685]
[552]
[697]
[552]
[552]
[416]
[552]
[420]
[416]
[154]
[420]
[154

[682]
[546]
[691]
[549]
[546]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[309]
[262]
[344]
[268]
[262]
[176]
[268]
[176]
[176]
[161]
[176]
[161]
[161]
[47]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[309]
[262]
[344]
[268]
[262]
[176]
[268]
[173]
[176]
[161]
[173]
[161]
[161]
[47]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[306]
[258]
[344]
[268]
[258]
[176]
[268]
[173]
[176]
[161]
[173]
[161]
[161]
[47]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[309]
[262]
[344]
[268]
[262]
[176]
[268]
[173]
[176]
[161]
[173]
[161]
[161]
[47]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[312]
[262]
[347]
[271]
[262]
[176]
[271]
[173]
[176]
[161]
[173]
[161]
[161]
[47]
[678]
[546]
[691]
[549]
[546]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[318]
[265]
[347]
[271]
[265]
[176]
[271]
[173]
[176]
[161]
[173]
[161]
[161]
[47]
[678]
[546]
[691]
[549]
[546]
[420]
[549]
[420]
[420]
[161]
[420]
[161

[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[157]
[420]
[157]
[309]
[262]
[347]
[271]
[262]
[176]
[271]
[173]
[176]
[157]
[173]
[157]
[157]
[44]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[161]
[420]
[161]
[306]
[262]
[347]
[271]
[262]
[176]
[271]
[173]
[176]
[161]
[173]
[161]
[161]
[44]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[161]
[420]
[161]
[309]
[262]
[347]
[271]
[262]
[176]
[271]
[173]
[176]
[161]
[173]
[161]
[161]
[44]
[682]
[549]
[691]
[552]
[549]
[420]
[552]
[420]
[420]
[161]
[420]
[161]
[315]
[265]
[350]
[271]
[265]
[176]
[271]
[173]
[176]
[161]
[173]
[161]
[161]
[44]
[682]
[549]
[691]
[549]
[549]
[420]
[549]
[420]
[420]
[161]
[420]
[161]
[322]
[268]
[350]
[274]
[268]
[176]
[274]
[173]
[176]
[161]
[173]
[161]
[161]
[44]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[161]
[420]
[161]
[315]
[265]
[347]
[271]
[265]
[176]
[271]
[170]
[176]
[161]
[170]
[161]
[161]
[44]
[682]
[549]
[694]
[552]
[549]
[420]
[552]
[420]
[420]
[157]
[420]
[157

...

[675]
[546]
[685]
[549]
[546]
[416]
[549]
[416]
[416]
[164]
[416]
[164]
[341]
[277]
[337]
[271]
[277]
[183]
[271]
[176]
[183]
[164]
[176]
[164]
[164]
[47]
[675]
[546]
[685]
[549]
[546]
[416]
[549]
[416]
[416]
[164]
[416]
[164]
[350]
[281]
[344]
[274]
[281]
[183]
[274]
[176]
[183]
[164]
[176]
[164]
[164]
[50]
[678]
[546]
[685]
[549]
[546]
[416]
[549]
[416]
[416]
[164]
[416]
[164]
[341]
[277]
[341]
[274]
[277]
[183]
[274]
[173]
[183]
[164]
[173]
[164]
[164]
[47]
[675]
[546]
[685]
[549]
[546]
[416]
[549]
[416]
[416]
[164]
[416]
[164]
[347]
[281]
[341]
[274]
[281]
[183]
[274]
[176]
[183]
[164]
[176]
[164]
[164]
[50]
[675]
[546]
[685]
[549]
[546]
[416]
[549]
[416]
[416]
[164]
[416]
[164]
[331]
[271]
[334]
[271]
[271]
[180]
[271]
[176]
[180]
[164]
[176]
[164]
[164]
[47]
[678]
[546]
[685]
[549]
[546]
[416]
[549]
[420]
[416]
[164]
[420]
[164]
[325]
[268]
[334]
[268]
[268]
[180]
[268]
[176]
[180]
[164]
[176]
[164]
[164]
[47]
[678]
[546]
[688]
[549]
[546]
[420]
[549]
[420]
[420]
[161]
[420]
[161

[682]
[549]
[694]
[549]
[549]
[416]
[549]
[416]
[416]
[154]
[416]
[154]
[287]
[252]
[341]
[265]
[252]
[170]
[265]
[167]
[170]
[154]
[167]
[154]
[154]
[37]
[685]
[549]
[694]
[549]
[549]
[416]
[549]
[416]
[416]
[154]
[416]
[154]
[277]
[246]
[337]
[262]
[246]
[167]
[262]
[164]
[167]
[154]
[164]
[154]
[154]
[34]
[685]
[552]
[694]
[549]
[552]
[416]
[549]
[420]
[416]
[151]
[420]
[151]
[271]
[243]
[337]
[262]
[243]
[167]
[262]
[164]
[167]
[151]
[164]
[151]
[151]
[34]
[685]
[549]
[694]
[549]
[549]
[416]
[549]
[416]
[416]
[154]
[416]
[154]
[281]
[249]
[337]
[265]
[249]
[167]
[265]
[167]
[167]
[154]
[167]
[154]
[154]
[37]
[685]
[549]
[694]
[549]
[549]
[416]
[549]
[416]
[416]
[154]
[416]
[154]
[284]
[249]
[341]
[265]
[249]
[167]
[265]
[167]
[167]
[154]
[167]
[154]
[154]
[34]
[682]
[549]
[691]
[549]
[549]
[416]
[549]
[416]
[416]
[157]
[416]
[157]
[306]
[258]
[347]
[268]
[258]
[173]
[268]
[167]
[173]
[157]
[167]
[157]
[157]
[37]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[157]
[416]
[157

[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[347]
[277]
[356]
[277]
[277]
[183]
[277]
[176]
[183]
[164]
[176]
[164]
[164]
[47]
[672]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[347]
[277]
[356]
[277]
[277]
[183]
[277]
[176]
[183]
[164]
[176]
[164]
[164]
[50]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[341]
[277]
[356]
[277]
[277]
[180]
[277]
[176]
[180]
[164]
[176]
[164]
[164]
[47]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[176]
[180]
[164]
[176]
[164]
[164]
[47]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[176]
[180]
[164]
[176]
[164]
[164]
[47]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[416]
[416]
[164]
[416]
[164]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[173]
[180]
[164]
[173]
[164]
[164]
[47]
[675]
[546]
[688]
[549]
[546]
[416]
[549]
[416]
[416]
[161]
[416]
[161

[675]
[546]
[685]
[546]
[546]
[416]
[546]
[413]
[416]
[161]
[413]
[161]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[170]
[180]
[161]
[170]
[161]
[161]
[44]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[413]
[416]
[161]
[413]
[161]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[170]
[180]
[161]
[170]
[161]
[161]
[44]
[675]
[546]
[685]
[546]
[546]
[416]
[546]
[413]
[416]
[161]
[413]
[161]
[344]
[277]
[360]
[277]
[277]
[180]
[277]
[170]
[180]
[161]
[170]
[161]
[161]
[44]
[675]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[344]
[277]
[356]
[277]
[277]
[180]
[277]
[170]
[180]
[161]
[170]
[161]
[161]
[44]
[675]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161]
[341]
[277]
[356]
[277]
[277]
[180]
[277]
[170]
[180]
[161]
[170]
[161]
[161]
[44]
[672]
[546]
[685]
[546]
[546]
[413]
[546]
[413]
[413]
[161]
[413]
[161]
[347]
[281]
[356]
[277]
[281]
[180]
[277]
[173]
[180]
[161]
[173]
[161]
[161]
[44]
[678]
[549]
[688]
[549]
[549]
[416]
[549]
[416]
[416]
[161]
[416]
[161

[697]
[558]
[707]
[558]
[558]
[410]
[558]
[420]
[410]
[142]
[420]
[142]
[205]
[211]
[315]
[246]
[211]
[148]
[246]
[148]
[148]
[142]
[148]
[142]
[142]
[18]
[688]
[555]
[701]
[555]
[555]
[416]
[555]
[423]
[416]
[148]
[423]
[148]
[262]
[236]
[344]
[258]
[236]
[164]
[258]
[157]
[164]
[148]
[157]
[148]
[148]
[31]
